#### Table of contents
- Importing libraries
- Loading datasets
- Feature Engineering
- Cross validation
- Model training
- Model evaluation
- Summary

#### Importing libraries


In [9]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn import model_selection
from scipy.stats import norm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,RobustScaler,MinMaxScaler
from sklearn.metrics import mean_absolute_error,mean_squared_error
from scipy import stats
from sklearn.model_selection import StratifiedKFold
import warnings
from sklearn import preprocessing
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from catboost import CatBoostRegressor
import xgboost as xgb
import lightgbm as lgb
warnings.filterwarnings('ignore')
from sklearn.decomposition import PCA
from scipy.stats import boxcox
import os
%matplotlib inline
np.random.seed(365)

#### Loading datasets

In [10]:
#load datasets
data_path_train = r"C:\Users\User\Desktop\Blessing_AI\Free_AI_Classes_2023\Data\Housing_dataset_train.csv"
df_train = pd.read_csv(data_path_train)

data_path_test = r"C:\Users\User\Desktop\Blessing_AI\Free_AI_Classes_2023\Data\Housing_dataset_test.csv"
df_test = pd.read_csv(data_path_test)

#### Feature Engineering

##### Filling missing values

In [14]:
#Filling missing values
#fill missing values in location with new class
df_train["loc"] = df_train["loc"].fillna("Unknown")
df_test["loc"] = df_test["loc"].fillna("Unknown")

#fill missing values in title with new class
df_train["title"] = df_train["title"].fillna("Unknown")
df_test["title"] = df_test["title"].fillna("Unknown")

#group hpuse title based on mode number of bedroom
# Group by 'Category' and calculate the mode of 'Value' for each group
def group_feature_by_feature_based_on_mode(by_feature,feature,df):
    modes_values = []
    titles = df[by_feature].unique()
    for title in titles:
        new_df = df[df[by_feature] == title]
        mode_value =  new_df[feature].mode()[0]
        modes_values.append(mode_value)
    mode_dict = dict(zip(titles, modes_values))

    return mode_dict

#Fill missing values in bedroom by mode value of house title
mode_values = group_feature_by_feature_based_on_mode(by_feature = "title",feature="bedroom",df=df_train)
#fill missing values by mode house title
def fill_missing_by_mode(cols,mode_dict=mode_values):
    col1 = cols[0]
    col2 = cols[1]
    if pd.isnull(col2):
        return mode_dict[col1]
    else:
        return col2
df_train["bedroom"] = df_train[["title","bedroom"]].apply(fill_missing_by_mode,axis = 1)

#Fill missing values in bathroon by mode value of house title
mode_values = group_feature_by_feature_based_on_mode(by_feature = "title",feature="bathroom",df=df_train)
df_train["bathroom"] = df_train[["title","bathroom"]].apply(fill_missing_by_mode,axis = 1)

#Fill missing values in parking by mode value of house titl
mode_values = group_feature_by_feature_based_on_mode(by_feature = "title",feature="parking_space",df=df_train)
df_train["parking_space"] = df_train[["title","parking_space"]].apply(fill_missing_by_mode,axis = 1)

print(f"Total missing data in train data is {df_train.isnull().sum().sum()}")
print(f"Total missing data in test data  is {df_test.isnull().sum().sum()}")

Total missing data in train data is 0
Total missing data in test data  is 0


##### Creating new features

In [15]:
#create new feature to inducate geopolitical zone
geo_states = {"North_central":["Benue","Kogi", "Kwara", "Nasarawa", "Niger", "Plateau"],
"North_East":["Adamawa", "Bauchi", "Borno", "Gombe", "Taraba", "Yobe"],
"North_West":["Kaduna", "Katsina", "Kano", "Kebbi", "Sokoto", "Jigawa","Zamfara"],
"South_East":["Abia", "Anambra", "Ebonyi", "Enugu", "Imo"],
"South":["Akwa Ibom", "Bayelsa", "Cross River", "Delta", "Edo", "Rivers"],
"South_West":["Ekiti", "Lagos", "Osun", "Ondo", "Ogun", "Oyo"]}

def add_geo_zone(df_train):
        df_train["Geo_zone"] = df_train["loc"]
        df_train.loc[df_train["loc"].isin(geo_states["North_central"]),"Geo_zone"] = "North_central"
        df_train.loc[df_train["loc"].isin(geo_states["North_East"]),"Geo_zone"] = "North_East"
        df_train.loc[df_train["loc"].isin(geo_states["North_West"]),"Geo_zone"] = "North_West"
        df_train.loc[df_train["loc"].isin(geo_states["South_East"]),"Geo_zone"] = "South_East"
        df_train.loc[df_train["loc"].isin(geo_states["South"]),"Geo_zone"] = "South"
        df_train.loc[df_train["loc"].isin(geo_states["South_West"]),"Geo_zone"] = "South_West"
        return df_train
df_train = add_geo_zone(df_train = df_train)

##### Encoding categorical features

In [16]:
#squaring and logging output feature
df_train["price_log"] = np.log(df_train["price"] + 1)
df_train["price_sqrt"] = np.sqrt(df_train["price"])

In [17]:
#Encode house location based mean houe price ranking
#avergae pricing based on location
location_ranks = list(df_train.groupby(["loc"])["price"].mean().sort_values(ascending=False).index)
location_ranks_dict = {}
for i in location_ranks:
    location_ranks_dict[i] = location_ranks.index(i) + 1
print(location_ranks_dict)
# Use the map() function to encode the data
categories_train = df_train["loc"]
categories_test = df_test["loc"]
encoded_data_train = categories_train.map(location_ranks_dict)
encoded_data_test = categories_test.map(location_ranks_dict)
df_train["loc"] = encoded_data_train

{'Lagos': 1, 'Bayelsa': 2, 'Rivers': 3, 'Akwa Ibom': 4, 'Delta': 5, 'Ogun': 6, 'Cross River': 7, 'Anambra': 8, 'Edo': 9, 'Oyo': 10, 'Ondo': 11, 'Enugu': 12, 'Osun': 13, 'Unknown': 14, 'Ekiti': 15, 'Kano': 16, 'Imo': 17, 'Nasarawa': 18, 'Katsina': 19, 'Plateau': 20, 'Benue': 21, 'Adamawa': 22, 'Kwara': 23, 'Niger': 24, 'Gombe': 25, 'Taraba': 26, 'Kaduna': 27, 'Bauchi': 28, 'Kogi': 29, 'Yobe': 30, 'Jigawa': 31, 'Borno': 32, 'Abia': 33, 'Zamfara': 34, 'Sokoto': 35, 'Ebonyi': 36, 'Kebbi': 37}


In [18]:
#Encode house location based mean houe price ranking
#avergae pricing based on location
location_ranks = list(df_train.groupby(["title"])["price"].mean().sort_values(ascending=False).index)
location_ranks_dict = {}
for i in location_ranks:
    location_ranks_dict[i] = location_ranks.index(i) + 1

print(location_ranks_dict)
# Use the map() function to encode the data
categories_train = df_train["title"]
categories_test = df_test["title"]
encoded_data_train = categories_train.map(location_ranks_dict)
encoded_data_test = categories_test.map(location_ranks_dict)
df_train["title"] = encoded_data_train

{'Mansion': 1, 'Penthouse': 2, 'Detached duplex': 3, 'Unknown': 4, 'Townhouse': 5, 'Terrace duplex': 6, 'Semi-detached duplex': 7, 'Bungalow': 8, 'Flat': 9, 'Apartment': 10, 'Cottage': 11}


In [21]:
#Encode house geopolotical zone  based mean houe price ranking
#avergae pricing based on title
location_ranks = list(df_train.groupby(["Geo_zone"])["price"].mean().sort_values(ascending=False).index)
location_ranks_dict = {}
for i in location_ranks:
    location_ranks_dict[i] = location_ranks.index(i) + 1
print(location_ranks_dict)
# Use the map() function to encode the data
categories_train = df_train["Geo_zone"]
encoded_data_train = categories_train.map(location_ranks_dict)
df_train["Geo_zone"] = encoded_data_train

{'South': 1, 'South_West': 2, 'Unknown': 3, 'South_East': 4, 'North_central': 5, 'North_East': 6, 'North_West': 7}


In [22]:
#Converting bedroom,bathroom and parking space to discrete variables
df_train[["bedroom","bathroom","parking_space"]] = df_train[["bedroom","bathroom","parking_space"]].astype(int)
df_test[["bedroom","bathroom","parking_space"]] = df_test[["bedroom","bathroom","parking_space"]].astype(int)
df_train.head()

,ID,loc,title,bedroom,bathroom,parking_space,price,Geo_zone,price_log,price_sqrt,bed_bath_paking
0,3583,19,7,2,2,1,1149999.565,7,13.955273,1072.380327,5.0
1,2748,11,10,1,2,4,1672416.689,2,14.329781,1293.219505,7.0
2,9261,15,4,7,5,2,3364799.814,2,15.028879,1834.339067,14.0
3,2224,8,3,5,2,4,2410306.756,4,14.695265,1552.516266,11.0
4,10300,29,6,1,5,6,2600700.898,5,14.771292,1612.668874,12.0


In [23]:
#Adding total number of bedrooms,bathrooms and parking space
df_train["bed_bath_paking"] =  df_train["bedroom"] + df_train["bathroom"] + df_train["parking_space"]
df_test["bed_bath_paking"] =  df_test["bedroom"] + df_test["bathroom"] + df_test["parking_space"]

#### Cross Validation

In [48]:
#dictionary of models to be used
models = {"Linear_Regression":LinearRegression(),
          "Random_Forest":RandomForestRegressor(random_state=0),"XGboost":xgb.XGBRegressor(random_state=0),
          "Catboost":CatBoostRegressor(random_state=0,silent=True),"Gradient_boosting":GradientBoostingRegressor(random_state=0)}

#Calculate the number of bins using the Sturges method
bins = int(np.ceil(np.log2(len(df_train)) + 1))
#Bin the data using the Sturges method
binned_data = pd.cut(df_train["price"], bins=bins,labels=False)
df_train["Bin_value"] = binned_data

#Divid data into dependent and independent variables
X = df_train.drop(["Bin_value","price","price_log","ID","price_sqrt"],axis=1)
y = df_train["Bin_value"]
target = "price_log"
#skf = StratifiedShuffleSplit(n_splits=10, test_size=0.1, random_state=0)
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)

#Custom cross validation function
def run(models):
    #create empty dic for model and scores
    scores = {}
    for model in models.keys():
        scores[model] = [] #create empty lsit to store model scores at on each fold
    for name,model in models.items():
        print(f"Running -- {model}")
        print("-------------------------")
        for i,(train_index, test_index) in enumerate(skf.split(X, y)):
            xtrain, xvalid = X.iloc[train_index], X.iloc[test_index]
            ytrain, yvalid = df_train[target].iloc[train_index], df_train[target].iloc[test_index]
            model.fit(xtrain, ytrain)
            yvalid = np.exp(yvalid) - 1
            #make predictions on validation data
            preds_valid =  np.exp(model.predict(xvalid)) -1
            rmse = mean_squared_error(yvalid, preds_valid,squared=False)
            print(f"Fold {i} score : ", rmse)
            scores[name].append(rmse)
        print(f"{model} -- mean rmse {np.mean(scores[name])}")
        print()

    #take the mean of scores for every model
    for name, model in models.items():
        scores[name] = np.mean(np.array(scores[name]))
    model_names = scores.keys()
    model_scores = scores.values()
    results = {"Model":model_names,"Rmse_score":model_scores}
    results = pd.DataFrame.from_dict(results) #create dictionary of model and corresponding mean rmse score
    return results

In [49]:
results = run(models = models)

Running -- LinearRegression()
-------------------------
Fold 0 score :  672365.0482334284
Fold 1 score :  659569.4882421744
Fold 2 score :  663019.814060372
Fold 3 score :  689962.0846617826
Fold 4 score :  683470.9811140407
Fold 5 score :  677756.1085147084
Fold 6 score :  688451.3732423781
Fold 7 score :  669106.8144500032
Fold 8 score :  736291.0143240849
Fold 9 score :  735560.6236546115
LinearRegression() -- mean rmse 687555.3350497584

Running -- RandomForestRegressor(random_state=0)
-------------------------
Fold 0 score :  638413.8994211911
Fold 1 score :  523460.57968888927
Fold 2 score :  598448.5131817762
Fold 3 score :  603657.4798439238
Fold 4 score :  628132.3878249043
Fold 5 score :  567013.2614946527
Fold 6 score :  614438.3000945566
Fold 7 score :  590526.5155155818
Fold 8 score :  657957.4870324606
Fold 9 score :  555875.6422062557
RandomForestRegressor(random_state=0) -- mean rmse 597792.4066304192

Running -- XGBRegressor(base_score=None, booster=None, callbacks=Non

In [52]:
sorted_df = results.sort_values(by="Rmse_score",ascending=True)
sorted_df.head()

,Model,Rmse_score
4,Gradient_boosting,552989.412159
3,Catboost,555607.047884
2,XGboost,565757.634637
1,Random_Forest,597792.406630
0,Linear_Regression,687555.335050


Comment : Gradient performs best compared t other models

In [53]:
#save results
path = r"C:\Users\User\Desktop\Blessing_AI\Free_AI_Classes_2023\Results"
os.chdir(path)
sorted_df.to_csv("model_results.csv",index=False)

In [ ]:
#### Model Training -